In [1]:
import time
import pickle

import pandas as pd
import numpy as np

from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

#from scipy.sparse import vstack 

from validation.data import *
from validation.scoring import bubbleup_score, BubbleUpMixin

from src.model import StarSpace

import torch
import torch.nn as nn
from torch import optim

In [ ]:
SAMPLE_SIZE = 10000 #500000
SOC_LEVEL = 2
BUBBLE_UP = 2
PROD = False

In [ ]:
X_train, y_train = dot_train_data(SOC_LEVEL)
X_test, y_test, ids = indeed_test_data('../data/us/everything.csv', SAMPLE_SIZE, SOC_LEVEL, use_gcs=True)
if PROD == False:
    noprod_idx = get_soc_n(y_train.astype(str), 2) != 51
    X_train, y_train = X_train[noprod_idx], y_train[noprod_idx]

In [ ]:
X_train = X_train.reset_index(drop=True)

X_train.head()

In [ ]:
path = 'lmd-classify-dot/ss-models/weights_2'
fs = GCSFileSystem(project='labor-market-data')
with fs.open(path) as f:
    embeddings = pickle.load(f)

# with open('weights_%s' % str(100000), 'rb') as f:
#     embeddings = pickle.load(f)

embeddings = torch.FloatTensor(embeddings)
embeddings = nn.Embedding.from_pretrained(embeddings)

In [ ]:
path = 'lmd-classify-dot/ss-models/train_vocab_2'
fs = GCSFileSystem(project='labor-market-data')
with fs.open(path) as f:
    vocab = pickle.load(f)

In [ ]:
# with open('train_vocab_%s' % str(100000), 'rb') as f:
#     vocab = pickle.load(f)

In [ ]:
model = StarSpace(
    d_embed=100,
    vocabulary=vocab,
    k_neg = 10,
    input_embedding = embeddings)

In [ ]:
X_train_emb = np.empty([X_train.shape[0],100])
X_test_emb = np.empty([X_test.shape[0],100])

start_time = time.time()
for i,doc in enumerate(X_train):
    s = doc.replace('\t',' ').split(' ')
    X_train_emb[i] = model.embed_doc(s)
print(time.time() - start_time)

start_time = time.time()
for i,doc in enumerate(X_test):
    s = doc.replace('\t',' ').split(' ')
    X_test_emb[i] = model.embed_doc(s)
print(time.time() - start_time)

In [ ]:
class BubbleUpLogisticRegression(BubbleUpMixin, LogisticRegression):
    pass

In [ ]:
Bubbles = BubbleUpLogisticRegression(C=2., solver='lbfgs', class_weight='balanced', 
                                     multi_class="multinomial", n_jobs=-1).set_bubbles(BUBBLE_UP)

Bubbles.fit(X_train_emb,y_train)

In [ ]:
y_pred = Bubbles.predict(X_test_emb)

In [ ]:
accuracy_score(get_soc_n(y_test.astype(str), 2).astype(str), y_pred) 